In [1]:
import numpy as np
import random
import time
import pandas as pd
import operator
from collections import defaultdict
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt

In [2]:
# Create doubly-linked list data structure
class Node(object):
    # Singly linked node
    def __init__(self, value=None, next=None, prev=None):
        self.value = value
        self.next = next
        self.prev = prev

class doubly_linked_list(object):
    def __init__(self):
        self.head = None
        self.tail = None
        self.count = 0

    def append_item(self, value):
        # Append an item 
        new_item = Node(value, None, None)
        if self.head is None:
            self.head = new_item
            self.tail = self.head
        else:
            new_item.prev = self.tail
            self.tail.next = new_item
            self.tail = new_item
        self.count += 1
    
    def iter(self):
        # Iterate the list
        current = self.head
        while current:
            item_val = current.value
            current = current.next
            yield item_val

    def print_forward(self):
        for node in self.iter():
            print(node)   
        
    def search_item(self, val):
         for node in self.iter():
            if val == node:
                return True
            return False
     
    def delete(self, value):
        # Delete a specific item
        current = self.head
        node_deleted = False
        if current is None:
            node_deleted = False

        elif current.value == value:
            self.head = current.next
            if(self.head is not None):
                self.head.prev = None
            node_deleted = True

        elif self.tail.value == value:
            self.tail = self.tail.prev
            self.tail.next = None
            node_deleted = True

        else:
            while current:
                if current.value == value:
                    current.prev.next = current.next
                    current.next.prev = current.prev
                    node_deleted = True
                current = current.next

        if node_deleted:
            self.count -= 1
    
    def is_empty(self):
        if self.head is None:
            return True
        else:
            current = self.head
            return current.value

# Initialize left and right bucket
def initialize_buckets(Partition_A,Partition_B,left_bucket,right_bucket):
    gain_array = np.array([])
    for i in Partition_A:
        gain = 0
        connected_array = df.iloc[i-1][1]
        for j in connected_array:
            if int(j) in Partition_A:
                gain -= 1
            else:
                gain += 1
        gain_array = np.append(gain_array,gain)
        left_bucket[int(gain)].append_item(i)
        
    gain_array = np.array([])
    for i in Partition_B:
        gain = 0
        connected_array = df.iloc[i-1][1]
        for j in connected_array:
            if int(j) in Partition_B:
                gain -= 1
            else:
                gain += 1
        gain_array = np.append(gain_array,gain)
        right_bucket[int(gain)].append_item(i)
    return left_bucket, right_bucket

# Find vertex with maximum gain in buckets
def calculate_max_gain(buckets, max_degree):
    for i in range(max_degree,-max_degree-1,-1):
        if(buckets[i].is_empty()!=True):
            vertex_max_gain = buckets[i].is_empty()
            gain = i
            return int(vertex_max_gain), gain
        
# Calculate number of cuts
def calculate_num_cuts(Partition_A,Partition_B):
    cuts = 0
    edges = 0
    for i in Partition_A:
        connected_array = df.iloc[i-1][1]
        for j in connected_array:
            if int(j) not in Partition_A:
                cuts += 1
            edges += 1 
            
    for i in Partition_B:
        connected_array = df.iloc[i-1][1]
        for j in connected_array:
            if int(j) not in Partition_B:
                cuts += 1
            edges += 1 
    return cuts, edges

# Update buckets after vertex move 
def update_buckets(buckets_to_update,buckets, vertex):
    bucket = defaultdict()
    for i in buckets:
        bucket[i] = doubly_linked_list()
    gain_array = np.array([])
    for i in buckets_to_update:
        gain = 0
        connected_array = df.iloc[i-1][1]
        for j in connected_array:
            if int(j) in buckets_to_update:
                gain -= 1
            else:
                gain += 1
        gain_array = np.append(gain_array,gain)
        if i is not vertex:
            bucket[int(gain)].append_item(i)
    return bucket

# Fiduccia Mattheyses algorithm for local search
def Fiduccia_Mattheyses_LS():
    start = time.time()
    # Create a partioning (A,B)
    num_vertices = len(df)
    vertices = np.array([i for i in range(1,num_vertices+1)])
    cut = int(0.5 * num_vertices)
    np.random.shuffle(vertices)
    Partition_A = vertices[:cut]
    Partition_B = vertices[cut:] 

    cuts_tracker = np.array([]).reshape(0,3)
    fixed_vertices = []

    # Create left and right gain buckets
    max_degree = np.max(df['Number of connected vertices'])
    buckets = np.array([i for i in range(-max_degree,max_degree+1)])
    left_bucket = defaultdict()
    right_bucket = defaultdict()
    for i in buckets:
        left_bucket[i] = doubly_linked_list()
        right_bucket[i] = doubly_linked_list()

    # Initialize buckets and cuts tracker
    left_buckets,right_buckets = initialize_buckets(Partition_A,Partition_B,left_bucket,right_bucket)

    cuts, _ = calculate_num_cuts(Partition_A,Partition_B)
    state = np.array([cuts,Partition_A,Partition_B])
    cuts_tracker = np.vstack((cuts_tracker,state))
    
    while(len(fixed_vertices)<num_vertices):
        if(len(Partition_A)>=len(Partition_B)):
            # Calculate vertex with maximum gain
            vertex_max_gain,gain = calculate_max_gain(left_buckets,max_degree)

            # Update buckets and partitions
            left_buckets[gain].delete(vertex_max_gain)
            Partition_A = np.delete(Partition_A, np.where(Partition_A == vertex_max_gain))
            Partition_B = np.append(Partition_B,vertex_max_gain)
            right_buckets = update_buckets(Partition_B, buckets, vertex_max_gain)

            # Update fixed vertices and cuts tracker
            fixed_vertices.append(vertex_max_gain)
            cuts, _ = calculate_num_cuts(Partition_A,Partition_B)
            state = np.array([cuts,Partition_A,Partition_B])
            cuts_tracker = np.vstack((cuts_tracker,state))

        else:
            # Calculate vertex with maximum gain
            vertex_max_gain,gain = calculate_max_gain(right_buckets,max_degree)

            # Update buckets and partitions
            right_buckets[gain].delete(vertex_max_gain)
            Partition_B = np.delete(Partition_B, np.where(Partition_B == vertex_max_gain))
            Partition_A = np.append(Partition_A,vertex_max_gain)
            left_buckets = update_buckets(Partition_A, buckets,vertex_max_gain)

            # Update fixed vertices and cuts tracker
            fixed_vertices.append(vertex_max_gain)
            cuts, _ = calculate_num_cuts(Partition_A,Partition_B)
            state = np.array([cuts,Partition_A,Partition_B])
            cuts_tracker = np.vstack((cuts_tracker,state))
        print(len(fixed_vertices),end='\r')
    
    optimal_num_cuts = np.argmin(cuts_tracker[:,0]) 
    final_partition1 = cuts_tracker[optimal_num_cuts][1]
    final_partition2 = cuts_tracker[optimal_num_cuts][2]
    end = time.time()
    elapsed_time = end-start
    print("Time taken", elapsed_time)
    return [optimal_num_cuts,final_partition1,final_partition2]

In [3]:
# Loading the single planar graph of 500 vertices
data = defaultdict(list)
for line in open("Graph500.txt"):
    split_line=line.split()
    ID_vertex = split_line[0]
    num_connected_vertices  = split_line[2]
    ID_connected_vertices = split_line[3:]
    if (ID_vertex) not in data.keys():
        data[ID_vertex].append(int(num_connected_vertices))
        data[ID_vertex].append(ID_connected_vertices)
df = pd.DataFrame(data.values(),columns = ['Number of connected vertices','ID connected vertices'])

In [7]:
Fiduccia_Mattheyses_LS()

Time taken 61.89481997489929


[202,
 array([413, 364, 456,  35, 122, 109, 412,  71, 344, 312, 304, 386, 177,
         43, 192, 426, 491, 288, 475, 280, 406, 273,  20, 129, 214,  72,
        455, 294, 275, 446,  74, 120, 148, 310, 402, 237, 410, 308,  50,
        153, 156,  15, 114,  54, 137, 460, 284, 281, 329, 375, 159, 423,
         93, 485,  30,  48, 243, 443, 371, 317, 144, 188, 444, 113, 474,
        361, 436,  51, 141,  73, 154, 107, 396, 218, 345, 483, 145,  21,
        151, 205,  91, 459,  33, 461,  99, 115, 219, 112, 362,  95, 223,
         22, 194, 416, 388, 497, 471, 228, 404, 405, 353, 339, 146,  81,
        351, 453, 385, 191, 185,  42, 382, 163,  98,  39, 230, 216, 327,
        490,  68,  18, 181, 278, 499, 487, 376, 162, 252, 492, 407, 138,
         14,  75, 133,  62, 132, 283, 253, 337, 372, 445,   1, 226, 500,
        468, 495, 260,  84, 383, 176, 477,  16, 201,  70,  52, 231, 254,
        448, 251, 244,  83, 392, 307, 305,  28, 189, 103, 267, 359, 370,
        106, 101, 174, 245, 178, 210, 470, 15